In [1]:
 %load_ext sql

In [2]:
%sql sqlite:///library.db

'Connected: @library.db'

In [6]:
%%sql
PRAGMA foreign_keys = OFF;

DROP TABLE IF EXISTS CD;
DROP TABLE IF EXISTS CD_Detail;
DROP TABLE IF EXISTS Books;
DROP TABLE IF EXISTS Book_Detail;
DROP TABLE IF EXISTS Scientific_Journals;
DROP TABLE IF EXISTS SJ_Detail;
DROP TABLE IF EXISTS Magazines;
DROP TABLE IF EXISTS Magazine_Detail;
DROP TABLE IF EXISTS Items;
DROP TABLE IF EXISTS Personnels;
DROP TABLE IF EXISTS Users;
DROP TABLE IF EXISTS People;
DROP TABLE IF EXISTS Borrowing;
DROP TABLE IF EXISTS Request;
DROP TABLE IF EXISTS Request_by;
DROP TABLE IF EXISTS Event;
DROP TABLE IF EXISTS Event_Detail;
DROP TABLE IF EXISTS Hosted_by;
DROP TABLE IF EXISTS Joining;

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [7]:
%%sql

PRAGMA foreign_keys = ON;

/*****************
*	Items
*****************/
CREATE TABLE IF NOT EXISTS Items(
    id INT PRIMARY KEY,
    title VARCHAR(50),
    releaseDate DATE,
    availability VARCHAR(10),
    CHECK (availability = 'available' 
		OR availability = 'borrowed' 
		OR availability = 'coming')
);


CREATE TABLE IF NOT EXISTS CD(
	id INT PRIMARY KEY,
	ISRC VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS CD_Detail(
	ISRC VARCHAR(50) PRIMARY KEY,
	artist VARCHAR(50),
	studio VARCHAR(50),
    genre VARCHAR(30)
);

CREATE TABLE IF NOT EXISTS Magazines(
	id INT PRIMARY KEY,
	ISSN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Magazine_Detail(
	ISSN VARCHAR(50) PRIMARY KEY,
	publisher VARCHAR(50),
    genre VARCHAR(30)
);

CREATE TABLE IF NOT EXISTS Scientific_Journals(
	id INT PRIMARY KEY,
	ISSN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS SJ_Detail(
	ISSN VARCHAR(50) PRIMARY KEY,
	field VARCHAR(30),
	researcher VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Books(
	id INT PRIMARY KEY,
	ISBN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id) ON DELETE CASCADE
);

CREATE TABLE Book_Detail(
	ISBN VARCHAR(50) PRIMARY KEY,
	author VARCHAR(50),
    publisher VARCHAR(50),
	genre VARCHAR(30),
	type VARCHAR(7),
	CHECK (type = 'printed' 
		OR type = 'online')
);

/*****************
*	People
*****************/
CREATE TABLE IF NOT EXISTS People (
	pid INT PRIMARY KEY,
	firstName VARCHAR(50),
	lastName VARCHAR(50),
	email VARCHAR(50),
	phone VARCHAR(10),
    address VARCHAR(256)
);

CREATE TABLE IF NOT EXISTS Users (
	pid INT PRIMARY KEY,
	totalFine INT,
	FOREIGN KEY (pid) REFERENCES People(pid) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Personnels (
	pid INT PRIMARY KEY,
	position VARCHAR(30),
    salary REAL,
	FOREIGN KEY (pid) REFERENCES People(pid) ON DELETE CASCADE
);

/*****************
*	Event
*****************/
CREATE TABLE IF NOT EXISTS Event (
	eid INT PRIMARY KEY,
	event INT,
    room VARCHAR(30),
	eventDate DATE,
	eventTime TIME
);

CREATE TABLE IF NOT EXISTS Event_Detail (
    event INT PRIMARY KEY,
    name VARCHAR (50) ,
	type VARCHAR (30),
	description VARCHAR(300),
	audiences VARCHAR(30)
);


/****************
*	Request
*****************/
CREATE TABLE IF NOT EXISTS Request (
	rid INT PRIMARY KEY,
	description VARCHAR(300)
);

/*****************
*	Relationships
*****************/
CREATE TABLE IF NOT EXISTS Borrowing (
	id INT,
	uid INT,
	dateBorrowed DATE,
    dateReturned DATE,
	dueDate DATE,
	fine INT,
	PRIMARY KEY (id, uid),
	FOREIGN KEY (id) REFERENCES Items(id)
	ON DELETE CASCADE,
	FOREIGN KEY (id) REFERENCES Users(pid)
	ON DELETE CASCADE,
	CHECK (dateBorrowed < dueDate 
		OR dateBorrowed < dateReturned)
);

CREATE TABLE IF NOT EXISTS Request_by (
	rid INT,
	pid INT,
	PRIMARY KEY(rid,pid),
	FOREIGN KEY (rid) REFERENCES Request(rid)
	ON DELETE CASCADE,
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Hosted_by (
	eid INT,
	pid INT,
	PRIMARY KEY(eid,pid),
	FOREIGN KEY (eid) REFERENCES Event(eid)
	ON DELETE CASCADE,
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Joining (
	eid INT,
	pid INT,
	PRIMARY KEY(pid, eid),
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE,
	FOREIGN KEY (eid) REFERENCES Event(eid)
	ON DELETE CASCADE
);



 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [4]:
%%sql

/*****************
*	Triggers
*****************/
--When inserted to the Borrowing table, sets dateBorrowed and dueDate when not set
CREATE TRIGGER IF NOT EXISTS borrowDate AFTER INSERT ON Borrowing
FOR EACH ROW
WHEN (NEW.dateBorrowed IS NULL)
BEGIN 
    UPDATE Borrowing SET dateBorrowed = DATE('now') WHERE id = NEW.id;
    UPDATE Borrowing SET dueDATE = DATE('now','+14 day') WHERE dueDate IS NULL AND id = NEW.id;
END;


CREATE TRIGGER IF NOT EXISTS fine AFTER UPDATE ON Borrowing
FOR EACH ROW
WHEN (NEW.dateReturned IS NOT NULL AND NEW.dueDate IS NOT NULL AND NEW.dateReturned > NEW.dueDate)
BEGIN
    UPDATE Borrowing SET fine = JulianDay(NEW.dateReturned) - JulianDay(NEW.dueDate) WHERE id = NEW.id;
    UPDATE Users SET totalFine = totalFine + (JulianDay(NEW.dateReturned) - JulianDay(NEW.dueDate));
END;


 * sqlite:///library.db
Done.
Done.


[]

In [43]:
%%sql

/*****************
*	Insert to test trigger
*****************/

INSERT INTO People (pid, firstName, lastName, address, email, phone)
    VALUES (123,'Nikola', 'Maeda', '4306 Triumph st', 'nikobox', '323');
INSERT INTO Users (pid, totalFine) VALUES (123, 0);
INSERT INTO Items (id) VALUES (1);
INSERT INTO Books (id) VALUES (1);
INSERT INTO Borrowing(id, uid) VALUES (1,123);
UPDATE Borrowing SET dateReturned = DATE('2020-04-30') WHERE id = 1;

 * sqlite:///library.db
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [8]:
%%sql
/*****************
*	Insertion
*****************/
/*****************
*	For Items
*****************/

INSERT INTO Items (id, title, releaseDate, availability) 
    VALUES (1, 'Book1', '2017-10-01', 'coming'),
           (2, 'CD1', '2016-01-01', 'borrowed'),
           (3, 'Book2', '1998-09-25', 'available'),
           (4, 'Book3', '1984-10-01', 'available'),
           (5, 'Scientific_Journal1', '2001-06-29', 'available'),
           (6, 'CD2', '2005-07-07', 'coming'),
           (7, 'Scientific_Journal2', '2003-07-04', 'coming'),
           (8, 'Scientific_Journal3', '1998-06-16', 'available'),
           (9, 'Book4', '2007-06-08', 'coming'),
           (10, 'CD3', '2005-05-09', 'borrowed'),
           (11, 'Magazine1', '2010-08-14', 'coming'),
           (12, 'Scientific_Journal4', '2002-01-02', 'available'),
           (13, 'Magazine2', '2019-10-02', 'coming'),
           (14, 'Book5', '2017-10-01', 'coming'),
           (15, 'Scientific_Journal5', '2019-10-09', 'available'),
           (16, 'CD4', '2009-06-01', 'coming'),
           (17, 'CD5', '2014-02-22', 'coming'),
           (18, 'Book6', '2020-01-20', 'available'),
           (19, 'Magazine3', '2013-11-21', 'available'),
           (20, 'Magazine4', '2020-01-02', 'available'),
           (21, 'Scientific_Journal6', '2004-10-31', 'coming'),
           (22, 'Scientific_Journal7', '2002-10-23', 'available'),
           (23, 'Book7', '2013-10-10', 'borrowed'),
           (24, 'Magazine5', '2012-12-20', 'available'),
           (25, 'CD6', '1999-09-30', 'available'),
           (26, 'Magazine6', '2011-10-06', 'coming'),
           (27, 'Book8', '1987-05-22', 'available'),
           (28, 'Magazine7', '1998-11-04', 'available'),
           (29, 'Book9', '2017-12-25', 'borrowed'),
           (30, 'Scientific_Journal8', '1999-03-02', 'available'),
           (31, 'Book10', '1960-03-10', 'coming'),
           (32, 'CD7', '1988-08-15', 'coming'),
           (33, 'CD8', '2017-10-01', 'available'),
           (34, 'Magazine8', '2009-03-19', 'coming'),
           (35, 'Scientific_Journal9', '2010-08-23', 'available'),
           (36, 'Magazine9', '2000-12-01', 'borrowed'),
           (37, 'CD9', '2001-04-21', 'available'),
           (38, 'CD10', '2002-08-19', 'available'),
           (39, 'Magazine10', '2017-10-10', 'borrowed'),
           (40, 'Scientific_Journal10', '1999-12-12', 'borrowed');
        

INSERT INTO CD (id, ISRC) 
    VALUES (2, 'US-B11-13-00005'),
           (6, 'US-B20-13-00004'),
           (10, 'US-B01-13-00002'),
           (16, 'US-B14-99-00105'),
           (17, 'UK-B07-13-00001'),
           (25, 'US-B09-13-00011'),
           (32, 'UK-B05-01-00234'),
           (33, 'UK-B03-13-00033'),
           (37, 'UK-B11-03-00123'),
           (38, 'US-B21-15-00012');

INSERT INTO CD_Detail (ISRC, artist, studio, genre) 
    VALUES ('US-B11-13-00005','Billy Bob', 'Studio1', 'Rock'),
           ('US-B20-13-00004','Artist Two', 'Studio2', 'R&B'),
           ('US-B01-13-00002','Billy Bab', 'Studio3', 'R&B'),
           ('US-B14-99-00105','Antonio Kim', 'Studio4', 'Rock'),
           ('UK-B07-13-00001','Nicholas Mang', 'Studio5', 'R&B'),
           ('US-B09-13-00011','Banana Man', 'Studio6', 'Rock'),
           ('UK-B05-01-00234','Eve Mina', 'Studio7', 'Rock'),
           ('UK-B03-13-00033','Mary Lee', 'Studio8', 'R&B'),
           ('UK-B11-03-00123','Artist Nine', 'Studio9', 'R&B'),
           ('US-B21-15-00012','Ocean Mama', 'Studio10', 'R&B');

INSERT INTO Magazines (id, ISSN) 
    VALUES (11, '1234-4321'),
           (13, '1271-4391'),
           (19, '1294-4102'),
           (20, '1234-3105'),
           (24, '0713-0001'),
           (26, '0913-7811'),
           (28, '0501-9023'),
           (34, '0313-8233'),
           (36, '1103-0123'),
           (39, '2115-1277');  

INSERT INTO Magazine_Detail (ISSN, publisher, genre) 
    VALUES ('1234-4321','Publisher1','Fashion'),
           ('1271-4391','Publisher2','Sports'),
           ('1294-4102','Publisher3','Video Games'),
           ('1234-3105','Publisher4','Cooking'),
           ('0713-0001','Publisher5','Sports'),
           ('0913-7811','Publisher6','Gardening'),
           ('0501-9023','Publisher7','Video Games'),
           ('0313-8233','Publisher8','Video Games'),
           ('1103-0123','Publisher9','Fashion'),
           ('2115-1277','Publisher10','Video Games'); 
        

INSERT INTO Scientific_Journals (id, ISSN) 
    VALUES (5, '4321-4321'),
           (7, '4391-2036'),
           (8, '1354-4102'),
           (12, '2235-3445'),
           (15, '3875-4968'),
           (21, '1357-5555'),
           (22, '1232-9023'),
           (30, '1123-2069'),
           (35, '1112-5040'),
           (40, '5995-0044'); 

INSERT INTO SJ_Detail (ISSN, field, researcher) 
    VALUES ('4321-4321', 'Agriculture', 'Angus Kan'),
           ('4391-2036', 'Physics', 'Billy Jeans'),
           ('1354-4102', 'Physics', 'James Chin'),
           ('2235-3445', 'Chemistry', 'Pam Beasley'),
           ('3875-4968', 'Agriculture', 'Michael Wang'),
           ('1357-5555', 'Chemistry', 'Dwight Schrute'),
           ('1232-9023', 'Agriculture', 'Dean Yonata'),
           ('1123-2069', 'Chemistry', 'Meme Man'),
           ('1112-5040', 'Chemistry', 'Robbie Kailley'),
           ('5995-0044', 'Botany', 'Robert Key'); 

INSERT INTO Books (id, ISBN) 
    VALUES (5, '978-90-55687-02-5'),
           (7, '111-92-92545-12-4'),
           (8, '264-02-19075-03-6'),
           (12, '853-98-88559-25-5'),
           (15, '886-10-10293-88-3'),
           (21, '091-12-57483-70-5'),
           (22, '102-88-13579-93-0'),
           (30, '345-23-25468-94-2'),
           (35, '398-45-85643-19-5'),
           (40, '767-04-31288-59-1');
        
INSERT INTO Book_Detail (ISBN, author, publisher, genre, type) 
    VALUES ('978-90-55687-02-5', 'Erwin Frogo', 'Publisher1', 'Mystery', 'online'),
           ('111-92-92545-12-4', 'Jenkins Kenker', 'Publisher2', 'Romance', 'online'),
           ('264-02-19075-03-6', 'Mem Loot', 'Publisher3', 'Mystery', 'printed'),
           ('853-98-88559-25-5', 'Rooty Rao', 'Publisher4', 'Mystery', 'printed'),
           ('886-10-10293-88-3', 'Nongo Bongo', 'Publisher5', 'Romance', 'online'),
           ('091-12-57483-70-5', 'Chingus Goo', 'Publisher6', 'Romance', 'online'),
           ('102-88-13579-93-0', 'Bumbo Bag', 'Publisher7', 'Fantasy', 'online'),
           ('345-23-25468-94-2', 'Mama Mio', 'Publisher8', 'Adventure', 'online'),
           ('398-45-85643-19-5', 'Jason Lim', 'Publisher9', 'Adventure', 'printed'),
           ('767-04-31288-59-1', 'Lovu Yoo', 'Publisher10', 'Romance', 'printed');

INSERT INTO People (pid, firstName, lastName, email, phone, address)
    VALUES (1, 'Ben', 'Anderson', 'bena89@gmail.com', '604-301-3215', '541 Robson St, Vancouver, BC'),
           (2, 'Ben', 'Anderson', 'bena89@gmail.com', '604-301-3215', '541 Robson St, Vancouver, BC'),
           (3, 'Ben', 'Anderson', 'bena89@gmail.com', '604-301-3215', '541 Robson St, Vancouver, BC'),
           (4, 'Ben', 'Anderson', 'bena89@gmail.com', '604-301-3215', '541 Robson St, Vancouver, BC'),
           (5, 'Ben', 'Anderson', 'bena89@gmail.com', '604-301-3215', '541 Robson St, Vancouver, BC'),
           (6, 'Ben', 'Anderson', 'bena89@gmail.com', '604-301-3215', '541 Robson St, Vancouver, BC');

/*INSERT INTO Users (pid, totalFine)
    VALUES ()

INSERT INTO Personnels (pid, position, salary)
    VALUES ()*/

 * sqlite:///library.db
Done.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.
6 rows affected.
0 rows affected.


[]

In [111]:
%%sql
select * FROM CD


 * sqlite:///library.db
Done.


id,ISRC
2,US-B11-13-00005
6,US-B20-13-00004
1000,US-B01-13-00002
16,US-B14-99-00105
17,UK-B07-13-00001
25,US-B09-13-00011
32,UK-B05-01-00234
33,UK-B03-13-00033
37,UK-B11-03-00123
38,US-B21-15-00012


In [46]:
%%sql
select * from Users;


 * sqlite:///library.db
Done.


pid,totalFine
123,10
